In [387]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep
import pandas as pd
import time

In [305]:
def properText(string):
    return ' '.join(list(map(lambda x: x.capitalize(),string.split(' '))))

def checkItemList(list1, list2):
    result = True
    if len(list1) == 0 or list1 == ['']:
        return (result, 0)
    else:    
        for i in list1:
            if i not in list2:
                result = False
                break
    return (result, i)

In [388]:
gecko_path = '/usr/local/bin/geckodriver'
url = 'https://www.kiva.org/lend?page=1'
url = 'https://www.kiva.org/lend?country=vn&sector=10&status=all'
countryListFile = 'countries.csv'
sectorListFile = 'sectors.csv'
attributeListFile = 'attributes.csv'
options = webdriver.firefox.options.Options()
options.headless = False
driver = webdriver.Firefox(options = options, executable_path = gecko_path)
driver.get(url)
time.sleep(1)
acceptButton = driver.find_element_by_xpath("//button[@id = 'onetrust-accept-btn-handler']")
acceptButton.click()
filterButton = driver.find_element_by_xpath("/html/body/div[4]/div[3]/div/div/div[1]/div[2]/div[2]/div[1]/a")
filterButton.click()

In [381]:
options = webdriver.firefox.options.Options()
options.headless = False
driver = webdriver.Firefox(options = options, executable_path = gecko_path)
driver.get(url)
time.sleep(1)

In [382]:
acceptButton = driver.find_element_by_xpath("//button[@id = 'onetrust-accept-btn-handler']")
acceptButton.click()
filterButton = driver.find_element_by_xpath("/html/body/div[4]/div[3]/div/div/div[1]/div[2]/div[2]/div[1]/a")
filterButton.click()

In [354]:
# genderOption = '1'
# keyWords = 'poor agriculture'
# selectedCountryNames = ['Vietnam','Burkina Faso']
# selectedSectorNames = ['Agriculture','Retail']
# selectedAttributeNames = ['Green','Youth']
# loanLength = '16 mths or less'
# loanStatus = 'funded' #'fundRaising','all'

In [367]:
loanURLs = []
# Select Gender Filter
if genderOption in genderDict.keys():
    gender = driver.find_element_by_xpath("//div[@class = 'gender-button-box triple-state-buttons']").\
            find_element_by_xpath(".//*[text() = '{0}']".format(genderDict[genderOption]))
    gender.click()
time.sleep(1)
    
# Select Borrower Type Filter
if typeOption in typeDict.keys():
    type = driver.find_element_by_xpath("//div[@class = 'isGroup-button-box triple-state-buttons']").\
            find_element_by_xpath(".//*[text() = '{0}']".format(typeDict[typeOption]))
    type.click()
time.sleep(1)

# Input keywords
driver.find_element_by_xpath("//input[@id = 'filter-keywords-search-box']").send_keys(keyWords)
time.sleep(1)

# Select countries
driver.find_element_by_xpath("//a[@data-reveal-id = 'countrySelectModal']").click()
time.sleep(2)
if len(selectedCountryNames) > 0:
    for i in range(len(selectedCountryNames)):
#         checkBoxList = [driver.find_element_by_xpath("//input[@class = 'countryCheckbox'and @value = '{0}']".format(countryCode)) for countryCode in countryCodes]
        checkBoxList = [driver.find_element_by_xpath("//div[contains(., '{0}')][@class = 'country filter checkbox-input']".format(countryName)) for countryName in selectedCountryNames]
        time.sleep(3)
        checkBoxList[i].find_element_by_xpath(".//input[@class = 'countryCheckbox']").click()
        time.sleep(3)
driver.find_element_by_id("filter-country-submit").click()
time.sleep(1)


# Select loan sectors
if len(selectedSectorNames) > 0:
    for i in range(len(selectedSectorNames)):
        checkBoxList = [driver.find_element_by_xpath("//li[contains(., '{0}')][@class = 'filter checkbox-input']".format(sectorName)) for sectorName in selectedSectorNames]
        time.sleep(3)
        checkBoxList[i].find_element_by_xpath(".//input[@name = 'filter-sector']").click()
        time.sleep(3)
time.sleep(1)

        
# Select loan attributes
if len(selectedAttributeNames) > 0:
    for i in range(len(selectedAttributeNames)):
        checkBoxList = [driver.find_element_by_xpath("//li[contains(., '{0}')][@class = 'filter checkbox-input']".format(attributeName)) for attributeName in selectedAttributeNames]
        time.sleep(3)
        checkBoxList[i].find_element_by_xpath(".//input[@name = 'filter-theme']").click()
        time.sleep(3)
time.sleep(1)


# Select loan length
if loanLength in lengthDict.keys():
    loanLengthDiv = driver.find_element_by_xpath("//div[@class = 'lenderTerm-button-box quintuple-state-buttons']")
    loanLengthDiv.find_element_by_xpath(".//label[text() = '{0}']".format(lengthDict(loanLength))).click()
time.sleep(1)


# Select loan status
if statusOption in statusDict.keys():
    driver.find_element_by_id('filter-loan-status-dropdown').click()
    time.sleep(1)
    driver.find_element_by_xpath(".//option[@value = '{0}']".format(statusDict[statusOption])).click()
    time.sleep(3)
time.sleep(1)

# Select loan URLs on pages
pageno = 1
while True:
    try:
        loanURLs += [x.get_attribute('href') for x in driver.find_elements_by_xpath("//a[@class = 'loan-card-2-borrower-name']")]
        print('Page no. {0} completed'.format(pageno))
        nextPage = driver.find_element_by_xpath("//a[@class = 'next button secondary ']")        
        pageno += 1
        nextPage.click()
        time.sleep(2)
    except Exception as e2:
        print("End of selected pages")
        break

if len(loanURLs) == 0:
    print('No loan found!')
else:
    print('{0} URLs collected'.format(len(loanURLs)))


Page no. 1 completed
End of selected pages
No loan found!


In [356]:
# User's paremeters input
## Read parameter files
genderDict = {'1': 'Women','2':'Men'}
typeDict = {'1': 'Individual','2':'Group'}
statusDict = {'1': 'Fundraising','2':'Funded'}
lengthDict = {'1': '8 mths or less','2':'16 mths or less','3': '2 yrs or less','4': '2 yrs or more'}
dfCountryList = pd.read_csv(countryListFile).astype(str)
dfSectorList = pd.read_csv(sectorListFile).astype(str)
dfAttributeList = pd.read_csv(attributeListFile).astype(str)

## Countries
print("Please select ID of countries on the following list. For all countries, please press Enter")
print(dfCountryList.to_string(index=False))
print("Example: 1,2")
time.sleep(1)
selectedCountryIDs = list(input("Loan countries: ").replace(' ','').split(','))
while not checkItemList(selectedCountryIDs,dfCountryList.ID.tolist())[0]:
    print(checkItemList(selectedCountryIDs,dfCountryList.ID.tolist())[1], " is not in the country list to select, please input again")
    selectedCountryIDs = list(input("Loan countries: ").replace(' ','').split(','))
selectedCountryNames = list(dfCountryList[dfCountryList.ID.isin(selectedCountryIDs)].Country)

## Sectors
print("Please select ID of sectors on the following list. For all sectors, please press Enter")
print(dfSectorList.to_string(index=False))
print("Example: 1,2,3")
time.sleep(1)
selectedSectorIDs = list(input("Loan sectors: ").replace(' ','').split(','))
while not checkItemList(selectedSectorIDs,dfSectorList.ID.tolist())[0]:
    print(checkItemList(selectedSectorIDs,dfSectorList.ID.tolist())[1], " is not in the sector list to select, please input again")
    selectedSectorIDs = list(input("Loan Sectors: ").replace(' ','').split(','))
selectedSectorNames = list(dfSectorList[dfSectorList.ID.isin(selectedSectorIDs)].Sector)

## Loan purpose
print("Please select ID of loan purposes on the following list. For all countries, please press Enter")
print(dfAttributeList.to_string(index=False))
print("Example: 1,3,4,5")
time.sleep(1)
selectedAttributeIDs = list(input("Loan Purposes: ").replace(' ','').split(','))
while not checkItemList(selectedAttributeIDs,dfAttributeList.ID.tolist())[0]:
    print(checkItemList(selectedAttributeIDs,dfAttributeList.ID.tolist())[1], " is not in the purpose list to select, please input again")
    selectedAttributeIDs = list(input("Loan Purposes: ").replace(' ','').split(','))
selectedAttributeNames = list(dfAttributeList[dfAttributeList.ID.isin(selectedAttributeIDs)].Attribute)
    
## Borrower Gender
print("\nPlease select ID of borrower's gender. For all genders, please press Enter")
print("1 Women\n2 Men")
genderOption = input("Borrower's gender option:")
while genderOption not in ['1','2','']:
    print(genderOption, 'is not in the list, please select again!')
    genderOption = input("Borrower's gender option:")
    
## Borrower Gender
print("\nPlease select ID of loan type (individual or group). For all loan types, please press Enter")
print("1 Individual\n2 Group")
typeOption = input("Loan type option:")
while typeOption not in ['1','2','']:
    print(typeOption, 'is not in the list, please select again!')
    typeOption = input("Loan type option:")
    
## Keywords input
keyWords = input('Please input keywords to search loans: ')

## Loan status
print("\nPlease select ID of loan status (fundraising or funded). For all loan statuses, please press Enter")
print("1 Fundraising\n2 Funded")
statusOption = input("Loan status option:")
while statusOption not in ['1','2','']:
    print(statusOption, 'is not in the list, please select again!')
    statusOption = input("Loan status option:")
    
## Loan length
print("\nPlease select ID of loan length. For all loan length values, please press Enter")
print("1 8 months or less\n2 16 months or less\n3 2 years or less\n4 2 years or more")
loanLength = input("Loan length option:")
while loanLength not in ['1','2','3','4','']:
    print(loanLength, 'is not in the list, please select again!')
    loanLength = input("Loan length option:")




Please select ID of countries on the following list. For all countries, please press Enter
ID            Country
 0            Albania
 1            Armenia
 2         Bangladesh
 3       Burkina Faso
 4            Bolivia
 5             Brazil
 6             Bhutan
 7             Belize
 8             Canada
 9        Congo (DRC)
10           Cameroon
11           Colombia
12         Costa Rica
13 Dominican Republic
14            Ecuador
15              Egypt
16               Fiji
17            Georgia
18              Ghana
19          Guatemala
20               Guam
21           Honduras
22              Haiti
23          Indonesia
24             Israel
25              India
26             Jordan
27              Kenya
28         Kyrgyzstan
29           Cambodia
30            Lao PDR
31            Lebanon
32            Liberia
33            Lesotho
34            Moldova
35         Madagascar
36               Mali
37    Myanmar (Burma)
38             Malawi
39             Mexico
40     

In [377]:
URLDir = 'url.csv'
loanURLs = ['babank.csv','alsdflsadf.csv']
with open(URLDir, 'w') as file:
    wr = csv.writer(file,dialect='excel')
    wr.writerows(loanURLs)

In [391]:
driver.find_elements_by_xpath("//div[@class = 'paging']")

[]